##Dependencias

In [1]:
import pandas as pd
import os 
from glob import glob

##Listar archivos

In [2]:
ruta = 'ibm_card_txn/*.csv'
archivos = glob(ruta)

In [3]:
type(archivos)

list

## Leer archivo base

In [9]:
df.columns

Index(['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip',
       'Merchant Name', 'Merchant City', 'Merchant State', 'Zip', 'MCC',
       'Errors?', 'Is Fraud?'],
      dtype='object')

In [4]:
cols = ['User', 'Card', 'Year', 'Month', 'Day', 'Time', 'Amount', 'Use Chip',
       'Merchant State','MCC',
       'Errors?', 'Is Fraud?']
names =['id_user','id_card','dt_timestamp','c_amt','d_use_chip','d_merchant_state'
        'id_mcc','d_errors','b_fraud']
df = pd.read_csv(archivos[0],dtype=str,usecols=cols)
df.shape

(100000, 12)

In [5]:
import pandas as pd

def validar_datos(df):
    # Convertir columnas numéricas con valores faltantes a 99999
    cols_numericas = ['User', 'Card', 'MCC']
    df[cols_numericas] = df[cols_numericas].apply(pd.to_numeric, errors='coerce').fillna(99999).astype(int)

    # Formatear Month y Day con dos dígitos
    df[['Month', 'Day']] = df[['Month', 'Day']].astype(int).astype(str).apply(lambda x: x.str.zfill(2))

    # Crear la columna de timestamp
    df['ts'] = pd.to_datetime(
        df['Year'].astype(str) + "-" + df['Month'] + "-" + df['Day'] + " " + df['Time'],
        format='%Y-%m-%d %H:%M'
    )
    
    # Convertir la columna 'Amount' a número, asegurando que es string antes de reemplazar '$'
    df['Amount'] = pd.to_numeric(df['Amount'].astype(str).str.replace('$', '', regex=False), errors='coerce')

    return df


In [6]:
validar_datos(df)

,User,Card,Year,Month,Day,Time,Amount,Use Chip,Merchant State,MCC,Errors?,Is Fraud?,ts
0,1101,6,2016,03,02,08:16,105.80,Online Transaction,NaN,4814,NaN,No,2016-03-02 08:16:00
1,1101,6,2016,03,02,12:25,27.63,Online Transaction,NaN,4121,NaN,No,2016-03-02 12:25:00
2,1101,6,2016,03,03,12:44,20.74,Swipe Transaction,CA,4121,NaN,No,2016-03-03 12:44:00
3,1101,6,2016,03,03,13:25,14.75,Swipe Transaction,CA,4121,NaN,No,2016-03-03 13:25:00
4,1101,6,2016,03,03,17:00,24.33,Swipe Transaction,CA,4121,NaN,No,2016-03-03 17:00:00
...,...,...,...,...,...,...,...,...,...,...,...,...,...
99995,1110,1,2013,07,10,14:21,123.58,Swipe Transaction,TX,5411,NaN,No,2013-07-10 14:21:00
99996,1110,1,2013,07,10,18:50,64.07,Swipe Transaction,TX,5812,NaN,No,2013-07-10 18:50:00
99997,1110,1,2013,07,11,18:50,47.54,Swipe Transaction,TX,5812,NaN,No,2013-07-11 18:50:00
99998,1110,1,2013,07,12,06:29,4.96,Swipe Transaction,TX,5411,NaN,No,2013-07-12 06:29:00


In [7]:
validar_datos(df).dtypes

User                       int64
Card                       int64
Year                      object
Month                     object
Day                       object
Time                      object
Amount                   float64
Use Chip                  object
Merchant State            object
MCC                        int64
Errors?                   object
Is Fraud?                 object
ts                datetime64[ns]
dtype: object

In [8]:
df['Errors?'].value_counts()

Errors?
Insufficient Balance                     918
Bad PIN                                  261
Technical Glitch                         186
Bad CVV                                   37
Bad Card Number                           37
Bad Expiration                            21
Bad Zipcode                               10
Insufficient Balance,Technical Glitch      6
Bad Expiration,Bad CVV                     1
Bad PIN,Insufficient Balance               1
Bad CVV,Insufficient Balance               1
Bad Zipcode,Technical Glitch               1
Bad Card Number,Bad Expiration             1
Name: count, dtype: int64

In [10]:
df['Is Fraud?'].value_counts(1)

Is Fraud?
No     0.99802
Yes    0.00198
Name: proportion, dtype: float64